In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import *

# Prima parte

## Circuito RC

Abbiamo effettuato le misurazioni con il cursore dell'oscilloscopio. \
**Resistenza interna oscilloscopio** = 50 $\Omega$

Intrvallo tensione (0.5,-0.5) V \
Capacità = 10 nF \
resistenza = 67.1 $k\Omega$ \
frequenza = 200 Hz \
incertezza su oscilloscopio (precisione strumento) = 8 mV


In [ ]:
# Primo plot dei dati
dati1 = pd.read_csv("Dati1.csv")

tempi1 = dati1["tempo(carica)"]
tens1 = dati1["tensione(carica1)"]

# plt.errorbar(tempi1, tens1, [8 for i in tens1], fmt = '.')
# plt.show()

In [ ]:
# prova fit dei dati con libreria
#def mod_RL(x,V,T,tau):
 #   return np.exp(-1*(x)/tau)*(2*V/(1 + np.exp(-1*T/(2*tau)))) 

def mod_exp(x, A, T, C):
    return A*np.exp(-x/T) + C

tempi1_ms = [i*1e-3 for i in tempi1]
tens1_mv = [i*1e-3 for i in tens1]
tens1_err = [.004 for i in tens1]

par1, val1, ers1, *other = LS_fit(tempi1_ms, tens1_mv, tens1_err, mod_exp, 
                                         A = 1, 
                                         T = 1, 
                                         C = 0,
                                         disp = True) 

#for p, v, e in zip(par1, val1, ers1):
#print(f"{p} = {v:.3f} ± {e:.3f}")
 
print(other[0]) 

In [ ]:
C_multim = 10e-9
C_multim_err = 0.1e-9

QC, tau_1, c_0 = val1
R_1 = 67.1e3 # Ohm
R_1_err = 0.1 # Ohm

C = tau_1/R_1
C_err = 1/R_1 * np.sqrt(ers1[1]**2 + (tau_1/R_1 * R_1_err)**2)

t_capacita = TestCompatibilita(C_multim, C, C_multim_err, C_err)

print(f"Capacità calcolata = ({C:.3g} ± {C_err:.3g}) F")
print(f"Test compatibilità: {t_capacita:.3g}")

In [ ]:
# Cella plot

x1 = np.linspace(min(tempi1_ms), max(tempi1_ms))
y1 = mod_exp(x1, *val1)

plt.errorbar(tempi1_ms, tens1_mv, tens1_err, label="Dati raccolti", fmt=".b")
plt.plot(x1, y1, label="Fit RC", color="tomato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.grid()
plt.legend()
plt.show()


## Circuito RL

induttanza ~ 10 mH \
resistenza = 1 $\text{k}\Omega$ \
frequenza = 700 Hz \
range di tensione = (-1,1) V \
precisione oscilloscopio = 20 mV

In [ ]:
# plot dei dati

dati2 = pd.read_csv("Dati2.csv")
tempi2 = dati2["tempo"]
tens2 = dati2["tensione(carica)"]

# plt.errorbar(tempi2, tens2, [20 for i in tens2])
# plt.show()

In [ ]:
# Fit circuito RL

tempi2_ms = [i*1e-3 for i in tempi2]
tens2_mv = [i*1e-3 for i in tens2]
tens2_err = [.02 for i in tens2]

par2, val2, ers2, *other2 = LS_fit(tempi2_ms, tens2_mv, tens2_err, mod_exp, 
                                          A = 1, 
                                          T = 1, 
                                          C = 1,
                                          disp = "8===D")

# for p, v, e in zip(par2, val2, ers2):
#     print(f"{p} = {v:.3g} ± {e:.3g}")

In [ ]:
Va, tau_2, c_1 = val2
R_2 = 1e3 # ohm
R_2_err = 0.1 # ohm

L = R_2 * tau_2
L_err = np.sqrt(tau_2**2 * R_2_err**2 + R_2**2 * ers2[1]**2)
print(f"Induttanza misurata = ({L:.3g} ± {L_err:.3g}) H")

In [ ]:
#plot dati RL



x1 = np.linspace(min(tempi2_ms), max(tempi2_ms))
y1 = mod_exp(x1, *val2)

plt.errorbar(tempi2_ms, tens2_mv, tens2_err, label="Dati raccolti", fmt=".b")
plt.plot(x1, y1, label="Fit RL", color="tomato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.grid()
plt.legend()
plt.show()

 # Seconda parte

## Circuito RLC

Resistenza = 300 $\Omega$ \
Induttanza e capacità uguali a prima \
Intervallo tensione (1,0)

Abbiamo stimato le costanti del sistema prima di prendere i dati:
* $\omega_0$ = 100 kHz
* $f_0 = \frac{\omega_0}{2\pi}$ = 16kHz 

Ci siamo resi presto conto che questi risultati sono insensati \
Frequenza effettivamente usata = 300 Hz


Per smorzamento critico abbiamo usato Resistenza = 3900 $\Omega$ \
Per sovrasmorzamento abbiamo usato Resistenza = 10_000 $\Omega$

**Abbiamo stimato inizialmente come errore 0.008. Sapendo che è arbitrario a causa della risoluzione dell'oscilloscopio abbiamo stimato l'errore a posteriori**

### Caso sottosmorzato

In [ ]:
#plot dati RLC sottosmorzato

dati3 = pd.read_csv("Dati3.csv")
tempi3=dati3["tempo"]
tensione3=dati3["tensione"]

tempi3_s = [i*1e-3 for i in tempi3]
tensione3_v = [i*1e-3 for i in tensione3]

tens3_err = [8 for i in tensione3]
tens3_err_s = [.01 for i in tensione3]   

# plt.errorbar(tempi3, tensione3_v, tens3_err_s, fmt = '.',label="Data points RLC")
# plt.title("Plot Circuito RLC sottosmorzato")
# plt.xlabel("Tempo [s]")
# plt.ylabel("Tensione [V]")
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
# fit dei dati nel caso sottosmorzato

R_3 = 300
R_3_err = 1
C_3 = C     #l'errore si chiama C_err
L_3 = L     #l'errore si chiama L_err

omega_0_calc = 1/np.sqrt(L_3*C_3)
gamma_calc_1 = R_3/(2*L_3)
omega_calc_1 = np.sqrt(omega_0_calc**2 - gamma_calc_1**2)

#calcolo errori
gamma_calc_1_err = np.sqrt((R_3_err/(2*L))**2 + (R_3*L_err/(2*L**2))**2)
omega_calc_1_err = np.sqrt((C*L_err)**2/(4*(C*L)**3) + (L*C_err)**2/(4*(C*L)**3))

def mod_ssm(x, I0, gamma, omega):
    return I0 * np.exp(-gamma*x)*np.sin(omega*x)

par3, val3, ers3, *other3 = LS_fit(tempi3_s, tensione3_v, tens3_err_s, mod_ssm, 
                                          I0 = 1, # inizializzo a 1 nel dubbio
                                          gamma = gamma_calc_1, 
                                          omega = omega_calc_1,
                                          disp = False)

# for p, v, e in zip(par3, val3, ers3):
#     print(f"{p} = {v:.3g} ± {e:.3g}")

In [ ]:
print("Analisi dati RLC sottosmorzato")
print("--------------------")
print(f"p-value = {float(other3[0]):.3g}")
print(f"Degrees of freedom = {other3[1]:.3g}")
print(f"chi2 = {other3[2]:.3g}")
print(f"chi2_red = {other3[2]/other3[1]:.3g}")
print("--------------------")

print(f"{par3[0]} = {val3[0]:.3g} ± {ers3[0]:.3g}")
print("--------------------")

print(f"{par3[1]} misurato = {val3[1]:.3g} ± {ers3[1]:.3g}")
print(f"valore teorico = {gamma_calc_1:.3g} ± {gamma_calc_1_err:.3g}")
t_ssm_gamma_1 = TestCompatibilita(val3[1], ers3[1], gamma_calc_1, gamma_calc_1_err) 
print(f"Test compatibilità: {t_ssm_gamma_1:.3g}")
print("Confronto con tabella ~ 95-98%")
print("--------------------")

print(f"{par3[2]} misurato = {val3[2]:.3g} ± {ers3[2]:.3g}")
print(f"valore teorico = {omega_calc_1:.3g} ± {omega_calc_1_err:.3g}")
t_ssm_omega_1 = TestCompatibilita(val3[2], ers3[2], omega_calc_1, omega_calc_1_err)
print(f"Test compatibilità: {t_ssm_omega_1:.3g}")
print("Confronto con tabella ~ ? %")
print("--------------------")

In [ ]:
#plot dati RLC sottosmorzato

x1 = np.linspace(min(tempi3_s), max(tempi3_s), 1000)
y1 = mod_ssm(x1, *val3)

plt.errorbar(tempi3_s, tensione3_v, tens3_err_s, label="Dati raccolti", fmt=".b")
plt.plot(x1, y1, label="RLC sottosmorzato", color="tomato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.grid()
plt.legend()
plt.show()

### Caso Smorzamento Critico 

In [ ]:
#plot dati RLC sottosmorzato
#resistenza usata R = 3900 omh

Dati3 = pd.read_csv("Dati3.csv")
tempo4 = Dati3["tempo2"].dropna()

tensione4 = Dati3["tensione2"].dropna()
tensione4_err = [8 for i in tensione4]

#trasporto le grandezze in SI
tempo4_s = [i*1e-3 for i in tempo4]
tensione4_v = [i*1e-3 for i in tensione4]
tensione4_err_v = [.008 for i in tensione4_err]

# plt.errorbar(tempo4_s,tensione4_v,tensione4_err_v, fmt = '.',label="Data points RLC")
# plt.title("Plot Circuito RLC smorzamento critico")
# plt.xlabel("Tempo [s]")
# plt.ylabel("Tensione [V]")
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
R_4 = 3900
R_4_err = 39

gamma_calc_2 = R_4/(2*L_3)

#calcolo errori
gamma_calc_2_err = np.sqrt((R_4_err/(2*L))**2 + (R_4*L_err/(2*L**2))**2)

#modello criticamente smorzato
def mod_csm(x, I0, gamma):
    return I0 * x * np.exp(-gamma*x)
#fit smorzamento critico

par4, val4, ers4, *other4 = LS_fit(tempo4_s, tensione4_v, tensione4_err_v, mod_csm, 
                                          I0 = 76000, 
                                          gamma = gamma_calc_2,
                                          disp = False)

# for p, v, e in zip(par4, val4, ers4):
#     print(f"{p} = {v:.3g} ± {e:.3g}")

In [ ]:
#plot per relazione RLC criticamente

x2 = np.linspace(min(tempo4_s), max(tempo4_s), 1000)
y2 = mod_csm(x2, *val4)

plt.errorbar(tempo4_s, tensione4_v, tensione4_err_v, label="Dati raccolti", fmt=".b")
plt.plot(x2, y2, label="RLC critico", color="tomato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.grid()
plt.legend()
plt.show()


In [ ]:
print("Analisi dati RLC smorzamento critico")
print("--------------------")
print(f"p-value = {other4[0]:.3g}")
print(f"Degrees of freedom = {other4[1]:.3g}")
print(f"chi2 = {other4[2]:.3g}")
print(f"chi2_red = {other4[2]/other4[1]:.3g}")
print("--------------------")

print(f"{par4[0]} = {val4[0]:.3g} ± {ers4[0]:.3g}")
print("--------------------")

print(f"{par4[1]} misurato = {val4[1]:.3g} ± {ers4[1]:.3g}")
print(f"valore teorico = {gamma_calc_2:.3g} ± {gamma_calc_2_err:.3g} ")
t_ssm_gamma_2 = TestCompatibilita(val4[1], ers4[1], gamma_calc_2, gamma_calc_2_err)
print(f"Test compatibilità: {t_ssm_gamma_2:.3g}")
print("Confronto con tabella ~ ? %")
print("--------------------")

In questo caso, probabilmente abbiamo mancato il picco di smorzamento critico, per cui la forma funzionale usata e' leggermente diversa da quella corretta

### Caso sovrasmorzato

In [ ]:
#plot dati RLC sovrasmorzato 
#resistenza utilizzato R= 10.000 ohm 

Dati5 = pd.read_csv("Dati3.csv")
tempo5= Dati5["tempo3"].dropna()
tensione5= Dati5["tensione3"].dropna()
tensione5_err = [8 for i in tensione5]

#trasporto le grandezze in SI
tempo5_s = [i*1e-3 for i in tempo5]
tensione5_v = [i*1e-3 for i in tensione5]
tensione5_err_v = [.02 for i in tensione5_err]

# plt.errorbar(tempo5_s,tensione5_v, tensione5_err_v, fmt = '.',label="Data points RLC")
# plt.title("Plot Circuito RLC sovrasmorzato")
# plt.xlabel("Tempo [s]")
# plt.ylabel("Tensione [V]")
# plt.legend()
# plt.grid()
# plt.show()

In [ ]:
R_5 = 10000
R_5_err = 100 
gamma_calc_3 = R_5/(2*L_3)
omega_calc_3 = np.sqrt(omega_0_calc**2 - gamma_calc_1**2 )

#calcolo errori
gamma_calc_3_err = np.sqrt((R_5_err/(2*L))**2 + (R_5*L_err/(2*L**2))**2)
omega_calc_3_err = np.sqrt((C*L_err)**2/(4*(C*L)**3) + (L*C_err)**2/(4*(C*L)**3))


# modello sovrasmorzato
def mod_svsm(x, I0, gamma, omega):
    return I0 * np.exp(-gamma*x) * (np.exp(omega*x) - np.exp(-omega*x))

#fit sovrasmorzamento
par5, val5, ers5, *other5 = LS_fit(tempo5_s, tensione5_v, tensione5_err_v, mod_svsm, 
                                          I0 = 1.15, 
                                          gamma = gamma_calc_3, 
                                          omega = omega_calc_3,
                                          disp = False)

# for p, v, e in zip(par5, val5, ers5):
#     print(f"{p} = {v:.3g} ± {e:.3g}")

In [ ]:
#plot per relazione RLC sovra

x3 = np.linspace(min(tempo5_s),max(tempo5_s),1000)
y3 = mod_svsm(x3, *val5)

plt.errorbar(tempo5_s, tensione5_v, tensione5_err_v, label="Dati raccolti", fmt=".b")
plt.plot(x3, y3, label="RLC sovrasmorzato", color="tomato")
plt.xlabel("Tempo [s]")
plt.ylabel("Tensione [V]")
plt.grid()
plt.legend()
plt.show()


In [ ]:
print("Analisi dati RLC sovrasmorzato")
print("--------------------")
print(f"p-value = {float(other5[0]):.3g}")
print(f"Degrees of freedom = {other5[1]:.3g}")
print(f"chi2 = {other5[2]:.3g}")
print(f"chi2_red = {other5[2]/other5[1]:.3g}")
print("--------------------")

print(f"{par5[0]} = {val5[0]:.3g} ± {ers3[0]:.3g}")
print("--------------------")

print(f"{par5[1]} misurato = {val5[1]:.3g} ± {ers3[1]:.3g}")
print(f"valore teorico = {gamma_calc_3:.3g} ± {gamma_calc_3_err:.3g} ")
t_ssm_gamma_3 = TestCompatibilita(val5[1], ers3[1], gamma_calc_3, gamma_calc_3_err) # 100 va ricalcolato
print(f"Test compatibilità: {t_ssm_gamma_3:.3g}")
print("Confronto con tabella ~ ? %")
print("--------------------")

print(f"{par5[2]} misurato = {val5[2]:.3g} ± {ers3[2]:.3g}")
print(f"valore teorico = {omega_calc_3:.3g} ± {omega_calc_3_err:.3g}")
t_ssm_omega_3 = TestCompatibilita(val5[2], ers3[2], omega_calc_3, omega_calc_3_err)
print(f"Test compatibilità: {t_ssm_omega_3:.3g}")
print("Confronto con tabella ~ ? %")
print("--------------------")